# Analyze Prediction Results
This notebook analyzes prediction results against ground truth data, calculates evaluation metrics, and visualizes the comparison.

## Import Required Libraries
Import libraries such as NumPy, pandas, matplotlib, and scikit-learn for data analysis and visualization.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

# Configure plots for better visualization
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)
sns.set(style='whitegrid')

## Load Prediction and Ground Truth Data
Load the prediction results and ground truth data from files or variables.

*Replace the placeholder data loading with your actual data.*

In [ ]:
# Example: Load data from CSV files (replace with your file paths)
try:
    predictions_df = pd.read_csv('ui/dash_app/predictions.csv') # Load predictions
    ground_truth_df = pd.read_csv('data/demand_prediction/cleaned_demand_data.csv') # Load ground truth

    # --- Data Preprocessing & Merging ---
    # Ensure timestamp and identifier columns are suitable for merging
    # Assuming predictions_df has 'timestamp', 'recipient_id', 'food_type', 'predicted_demand', 'predicted_disaster_flag'
    # Assuming ground_truth_df has 'timestamp', 'recipient_id', 'food_type', 'demand_kg', 'disaster_flag'

    # Rename columns for clarity before merge if necessary
    # predictions_df = predictions_df.rename(columns={'predicted_demand': 'demand_pred', 'predicted_disaster_flag': 'disaster_pred'})
    # ground_truth_df = ground_truth_df.rename(columns={'demand_kg': 'demand_true', 'disaster_flag': 'disaster_true'})

    # Define the key columns for merging
    merge_keys = ['timestamp', 'recipient_id', 'food_type'] # Add 'region' if it's in both and needed

    # Merge the dataframes
    data = pd.merge(predictions_df, ground_truth_df, on=merge_keys, how='inner')

    # --- Assign y_true and y_pred ---
    # For Regression (Demand Prediction)
    y_pred_demand = data['predicted_demand'] # Adjust column name if different
    y_true_demand = data['demand_kg']      # Adjust column name if different

    # For Classification (Disaster Flag Prediction)
    # Ensure the predicted flag column exists and handle potential missing values/types
    if 'predicted_disaster_flag' in data.columns:
        data['predicted_disaster_flag'] = data['predicted_disaster_flag'].fillna(0).astype(int) # Example handling
        y_pred_disaster = data['predicted_disaster_flag']
        y_true_disaster = data['disaster_flag']
        print("Disaster flag columns found for classification analysis.")
    else:
        print("Warning: 'predicted_disaster_flag' column not found in predictions. Skipping classification analysis.")
        y_pred_disaster = None
        y_true_disaster = None

    print("Data loaded and merged successfully.")

except FileNotFoundError:
    print("One or more CSV files not found. Please check file paths:")
    print("- ui/dash_app/predictions.csv")
    print("- data/demand_prediction/cleaned_demand_data.csv")
    # Fallback to sample data if files are missing (optional)
    print("\nUsing sample classification data for demonstration as fallback.")
    y_true_demand = np.array([2.5, 0.5, 2.1, 1.9, 0.8, 2.2, 0.3, 0.1, 2.8, 0.4] * 2) # Sample regression
    y_pred_demand = np.array([2.3, 0.7, 2.0, 1.5, 1.0, 2.1, 0.5, 0.2, 2.5, 0.6] * 2) # Sample regression
    y_true_disaster = np.array([1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0]) # Sample classification
    y_pred_disaster = np.array([1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1]) # Sample classification
    data = pd.DataFrame({'y_true_demand': y_true_demand[:14], 'y_pred_demand': y_pred_demand[:14], 'y_true_disaster': y_true_disaster, 'y_pred_disaster': y_pred_disaster})
except Exception as e:
    print(f"An error occurred during data loading/merging: {e}")
    # Handle other potential errors like missing columns
    data = pd.DataFrame() # Ensure data is an empty DataFrame on error

if not data.empty:
    print("\nGround Truth Demand Sample:")
    print(y_true_demand.head())
    print("\nPredicted Demand Sample:")
    print(y_pred_demand.head())
    if y_true_disaster is not None:
      print("\nGround Truth Disaster Flag Sample:")
      print(y_true_disaster.head())
      print("\nPredicted Disaster Flag Sample:")
      print(y_pred_disaster.head())
    print("\nCombined DataFrame Head:")
    print(data.head())
else:
    print("\nData loading failed or resulted in an empty DataFrame.")

## Calculate Evaluation Metrics
Compute metrics such as accuracy, precision, recall, F1-score, or mean squared error depending on the task.

*Choose the metrics appropriate for your task (classification or regression).*

In [ ]:
if not data.empty and y_true_disaster is not None and y_pred_disaster is not None:
    # --- Classification Metrics (Disaster Flag) ---
    print("--- Classification Metrics (Disaster Flag) ---")
    try:
        accuracy = metrics.accuracy_score(y_true_disaster, y_pred_disaster)
        precision = metrics.precision_score(y_true_disaster, y_pred_disaster, average='binary', zero_division=0)
        recall = metrics.recall_score(y_true_disaster, y_pred_disaster, average='binary', zero_division=0)
        f1 = metrics.f1_score(y_true_disaster, y_pred_disaster, average='binary', zero_division=0)
        conf_matrix = metrics.confusion_matrix(y_true_disaster, y_pred_disaster)
        class_report = metrics.classification_report(y_true_disaster, y_pred_disaster, zero_division=0)

        print(f"Accuracy: {accuracy:.4f}")
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1-Score: {f1:.4f}")
        print("\nConfusion Matrix:")
        print(conf_matrix)
        print("\nClassification Report:")
        print(class_report)
    except Exception as e:
        print(f"Could not calculate classification metrics: {e}")
else:
    print("Skipping classification metrics (data unavailable or loading failed).")
    conf_matrix = None # Ensure conf_matrix is defined for later visualization checks

if not data.empty:
    # --- Regression Metrics (Demand Prediction) ---
    print("\n--- Regression Metrics (Demand) ---")
    try:
        mse = metrics.mean_squared_error(y_true_demand, y_pred_demand)
        mae = metrics.mean_absolute_error(y_true_demand, y_pred_demand)
        rmse = np.sqrt(mse)
        r2 = metrics.r2_score(y_true_demand, y_pred_demand)
        print(f"Mean Squared Error (MSE): {mse:.4f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.4f}")
        print(f"Mean Absolute Error (MAE): {mae:.4f}")
        print(f"R-squared (R2): {r2:.4f}")
    except Exception as e:
        print(f"Could not calculate regression metrics: {e}")
else:
     print("Skipping regression metrics (data unavailable or loading failed).")

## Visualize Predictions vs Ground Truth
Create plots such as scatter plots, confusion matrices, or line plots to compare predictions with ground truth.

In [ ]:
# --- Classification Visualization: Confusion Matrix ---
if conf_matrix is not None:
    print("--- Confusion Matrix Visualization (Disaster Flag) ---")
    try:
        disp = metrics.ConfusionMatrixDisplay(confusion_matrix=conf_matrix)
        disp.plot(cmap=plt.cm.Blues)
        plt.title('Confusion Matrix (Disaster Flag)')
        plt.show()
    except Exception as e:
        print(f"Could not plot confusion matrix: {e}")
else:
    print("Skipping confusion matrix plot (matrix not available).")

# --- Regression Visualization: Scatter Plot ---
if not data.empty:
    print("\n--- Predictions vs. Ground Truth Scatter Plot (Demand) ---")
    try:
        plt.figure(figsize=(8, 8))
        plt.scatter(y_true_demand, y_pred_demand, alpha=0.6, edgecolors='k')
        # Add a diagonal line for reference (perfect prediction)
        min_val = min(y_true_demand.min(), y_pred_demand.min())
        max_val = max(y_true_demand.max(), y_pred_demand.max())
        plt.plot([min_val, max_val], [min_val, max_val], 'r--', lw=2, label='Ideal Prediction')
        plt.xlabel('Ground Truth Demand (kg)')
        plt.ylabel('Predicted Demand (kg)')
        plt.title('Demand: Predictions vs. Ground Truth')
        plt.legend()
        plt.grid(True)
        plt.axis('equal') # Ensure equal scaling for x and y axes
        plt.show()
    except Exception as e:
        print(f"Could not plot regression scatter plot: {e}")
else:
    print("Skipping regression scatter plot (data unavailable).")

## Analyze Errors
Identify and analyze cases where predictions deviate significantly from ground truth, and visualize these errors.

In [ ]:
if not data.empty:
    # --- Error Analysis for Classification (Disaster Flag) ---
    if y_true_disaster is not None and y_pred_disaster is not None:
        print("--- Disaster Flag Misclassification Analysis ---")
        try:
            misclassified_indices = np.where(y_true_disaster != y_pred_disaster)[0]
            misclassified_data = data.iloc[misclassified_indices]

            print(f"Number of misclassified disaster flag examples: {len(misclassified_data)}")
            if not misclassified_data.empty:
                print("\nMisclassified Disaster Flag Examples (Sample):")
                # Show relevant columns: timestamp, region, true flag, predicted flag
                print(misclassified_data[['timestamp', 'region', 'food_type', 'disaster_flag', 'predicted_disaster_flag']].head())
            # Further analysis could involve looking at features of misclassified points
        except Exception as e:
            print(f"Could not perform classification error analysis: {e}")
    else:
        print("Skipping classification error analysis (data unavailable).")

    # --- Error Analysis for Regression (Demand) ---
    print("\n--- Demand Prediction Error Analysis ---")
    try:
        # Calculate residuals (errors)
        errors = y_true_demand - y_pred_demand
        data['error'] = errors
        data['abs_error'] = np.abs(errors)

        print(f"Average Absolute Error: {data['abs_error'].mean():.4f}")
        print(f"Maximum Absolute Error: {data['abs_error'].max():.4f}")

        # Sort by absolute error to find largest deviations
        largest_errors = data.sort_values(by='abs_error', ascending=False)
        print("\nExamples with Largest Demand Prediction Errors (Top 5):")
        # Show relevant columns: timestamp, region, food_type, true demand, predicted demand, error
        print(largest_errors[['timestamp', 'region', 'food_type', 'demand_kg', 'predicted_demand', 'error', 'disaster_flag']].head())

        # Visualize residuals
        plt.figure(figsize=(10, 6))
        plt.scatter(y_pred_demand, errors, alpha=0.6, edgecolors='k')
        plt.axhline(0, color='red', linestyle='--', lw=2) # Line at zero error
        plt.xlabel('Predicted Demand (kg)')
        plt.ylabel('Residuals (True - Predicted)')
        plt.title('Residual Plot (Demand Prediction)')
        plt.grid(True)
        plt.show()

        # Histogram of errors
        plt.figure(figsize=(10, 6))
        sns.histplot(errors, kde=True, bins=30)
        plt.xlabel('Error (True - Predicted Demand)')
        plt.ylabel('Frequency')
        plt.title('Distribution of Demand Prediction Errors')
        plt.grid(True)
        plt.show()
    except Exception as e:
        print(f"Could not perform regression error analysis: {e}")
else:
    print("\nSkipping error analysis (data unavailable or loading failed).")